In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv("../input/car-car/train.csv")
test = pd.read_csv("../input/car-car/test.csv")
age = pd.read_csv("../input/car-car/age_gender_info.csv")
display(train,test,age)

In [ ]:
drop_index = train[train["자격유형"] == "O"].index
drop_index

In [ ]:
train = train.drop(drop_index)

In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
pd.options.display.max_columns = 9999
all_data = pd.merge(all_data, age, how = "left" ,on = "지역")

In [ ]:
cat = all_data.groupby("총세대수")[all_data.columns[all_data.dtypes == "object"]].nunique().add_prefix("nuni")
all_data = all_data.join(cat,on = "총세대수")

In [ ]:
car_fm =all_data.groupby("단지내주차면수")["전용면적별세대수"].agg(len).to_frame("car_fm")
all_data = all_data.join(car_fm, on = "단지내주차면수")

In [ ]:
region = all_data.groupby(["지역","임대건물구분"])["전용면적"].agg(["count","sum","mean","min","max","median","std"]).add_prefix("r_")
all_data = all_data.join(region, on = ["지역","임대건물구분"])

In [ ]:
type_per = all_data.groupby("단지코드")["자격유형"].nunique().to_frame("type_per")
all_data = all_data.join(type_per, on = "단지코드")

In [ ]:
all_data["va-bus"] = all_data["공가수"] * all_data["도보 10분거리 내 버스정류장 수"]

In [ ]:
all_data["traffic"] = np.where(((all_data["도보 10분거리 내 버스정류장 수"] == 6) | (all_data["도보 10분거리 내 지하철역 수(환승노선 수 반영)"] == 2)),1,0)

In [ ]:
all_data["type_va"] = np.where(((all_data["공급유형"] == "공공임대(10년)") & (all_data["공가수"] > 2)),1,0)

In [ ]:
b_type = all_data.groupby(["공급유형","지역"])["전용면적별세대수"].agg(["count","max"]).add_prefix("b_")
all_data = all_data.join(b_type, on = ["공급유형","지역"])

In [ ]:
pd.options.display.max_rows = 999
all_data.corr()["등록차량수"].sort_values(ascending=False)

In [ ]:
all_data2 = all_data.drop(columns=["등록차량수","단지코드"])
all_data2

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in all_data2.columns[all_data2.dtypes == "object"]:
    all_data2[i] = le.fit_transform(list(all_data2[i]))
all_data2

In [ ]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]

In [ ]:
result = 0
best_score = 0
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
kf = KFold(n_splits = 5, shuffle = True, random_state = 82)
for train_index, valid_index in kf.split(train2):
    X_train, X_valid = train2.iloc[train_index], train2.iloc[valid_index]
    y_train, y_valid = np.log(train["등록차량수"]).iloc[train_index], np.log(train["등록차량수"]).iloc[valid_index]
    cbr = CatBoostRegressor(verbose = 2000,eval_metric = "MAE",iterations = 1000000,learning_rate = 0.012911)
    cbr.fit(X_train,y_train,eval_set=(X_valid,y_valid),early_stopping_rounds=50,cat_features = [1,2,3,7])
    result += cbr.predict(test2) / 5
    best_score += cbr.best_score_["validation"]["MAE"] / 5
result

In [ ]:
best_score # 0.0178

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (16,8))
sns.displot((np.log(train["등록차량수"])))

In [ ]:
result_df = pd.DataFrame({"단지코드":test["단지코드"],"result":np.exp(result)})
a = result_df.groupby("단지코드")["result"].mean().reset_index().rename(columns={"단지코드":"code"})

In [ ]:
sub = pd.read_csv("../input/car-car/sample_submission.csv")
sub = pd.merge(sub, a , on = "code", how = "left")
sub = sub.drop(columns = "num")
sub.columns = ["code","num"]
sub

In [ ]:
sub.to_csv("sub0722_2.csv",index=False)